# Interactive maps with bokeh

### Line map
1. Read the data
2. calculate x and y coordinates
3. convert the DataFrame into a ColumnDataSource
4. make the map and save it

In [1]:
import geopandas as gpd

In [2]:
metro_fp = './shapely/dataE5/dataE5/metro.shp'
metro = gpd.read_file(metro_fp)

In [3]:
metro.head()

,NUMERO,SUUNTA,geometry
0,1300M,1,LINESTRING (2561676.997249531 6681346.00195433...
1,1300M,2,LINESTRING (2550919.001803585 6672692.00211347...
2,1300M1,1,LINESTRING (2561676.997249531 6681346.00195433...
3,1300M1,2,LINESTRING (2559946.003624604 6678095.99842650...
4,1300M2,1,LINESTRING (2559946.003624604 6678095.99842650...


In [28]:
def getPointCoords(row, geom, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y

In [4]:
def getLineCoords(row, geom, coord_type):
    """Returns a list of coordinates ('x' or 'y') of a LineString geometry"""
    if coord_type == 'x':
        return list( row[geom].coords.xy[0] )
    elif coord_type == 'y':
        return list( row[geom].coords.xy[1] )

In [7]:
metro['x'] = metro.apply(getLineCoords, geom='geometry', coord_type='x', axis=1)
metro['y'] = metro.apply(getLineCoords, geom='geometry', coord_type='y', axis=1)
metro.head()

,NUMERO,SUUNTA,geometry,x,y
0,1300M,1,LINESTRING (2561676.997249531 6681346.00195433...,"[2561676.997249531, 2560202.997150008, 2560127...","[6681346.001954339, 6681016.996685321, 6680969..."
1,1300M,2,LINESTRING (2550919.001803585 6672692.00211347...,"[2550919.001803585, 2551145.9991329825, 255126...","[6672692.002113477, 6672713.997145447, 6672737..."
2,1300M1,1,LINESTRING (2561676.997249531 6681346.00195433...,"[2561676.997249531, 2560202.997150008, 2560127...","[6681346.001954339, 6681016.996685321, 6680969..."
3,1300M1,2,LINESTRING (2559946.003624604 6678095.99842650...,"[2559946.003624604, 2560094.9990514405, 256018...","[6678095.998426503, 6678179.9976436375, 667824..."
4,1300M2,1,LINESTRING (2559946.003624604 6678095.99842650...,"[2559946.003624604, 2559644.0031698933, 255947...","[6678095.998426503, 6678008.998522878, 6677957..."


In [9]:
len(metro.x[0])

103

In [10]:
len(metro.x)

16

In [11]:
m_df = metro.drop('geometry', axis=1).copy()

In [13]:
from bokeh.plotting import figure, save
from bokeh.models import ColumnDataSource

msource = ColumnDataSource(m_df)

In [14]:
type(msource)

bokeh.models.sources.ColumnDataSource

In [16]:
p = figure(title="A map of the Helsinki metro")

p.multi_line('x', 'y', source=msource, color='red', line_width=3)

outfp = './shapely/dataE5/dataE5/metro_map.html'

save(p, outfp)

/home/dockeruser/anaconda3/lib/python3.6/site-packages/bokeh/io.py:419: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warnings.warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/home/dockeruser/anaconda3/lib/python3.6/site-packages/bokeh/io.py:429: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warnings.warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'/home/dockeruser/hostname/workspace/git/kaden/individual_project_20171130/shapely/dataE5/dataE5/metro_map.html'

### Polygon map with Points and Lines

In [17]:
from bokeh.plotting import figure, save
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper
import geopandas as gpd
import pysal as ps

In [18]:
grid_fp = r'./shapely/dataE5/dataE5/TravelTimes_to_5975375_RailwayStation.shp'
point_fp = r'./shapely/dataE5/dataE5/addresses.shp'
metro_fp = r'./shapely/dataE5/dataE5/metro.shp'

grid = gpd.read_file(grid_fp)
points = gpd.read_file(point_fp)
metro = gpd.read_file(metro_fp)

INFO:Fiona:Failed to auto identify EPSG: 7


In [20]:
CRS = grid.crs
print(CRS)

{'init': 'epsg:3067'}


In [21]:
points['geometry'] = points['geometry'].to_crs(crs=CRS)
metro['geometry'] = metro['geometry'].to_crs(crs=CRS)

In [23]:
points['geometry'].head(1)

0    POINT (385149.4478367225 6671962.669661295)
Name: geometry, dtype: object

In [24]:
metro['geometry'].head(1)

0    LINESTRING (395534.7026002127 6679490.08463068...
Name: geometry, dtype: object

In [25]:
grid['geometry'].head(1)

0    POLYGON ((382000.0001358641 6697750.000038058,...
Name: geometry, dtype: object

In [26]:
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )

In [29]:
# Get the Polygon x and y coordinates
grid['x'] = grid.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
grid['y'] = grid.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)

# Calculate x and y coordinates of the line
metro['x'] = metro.apply(getLineCoords, geom='geometry', coord_type='x', axis=1)
metro['y'] = metro.apply(getLineCoords, geom='geometry', coord_type='y', axis=1)

# Calculate x and y coordinates of the points
points['x'] = points.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
points['y'] = points.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

In [34]:
grid.x[0]

[382000.00013586413,
 381750.0001359122,
 381750.00013590575,
 382000.0001358568,
 382000.00013586413]

In [30]:
grid[['x', 'y']].head(2)

,x,y
0,"[382000.00013586413, 381750.0001359122, 381750...","[6697750.000038058, 6697750.000038066, 6698000..."
1,"[382250.0001358146, 382000.00013586413, 382000...","[6697750.000038053, 6697750.000038058, 6698000..."


In [31]:
metro[['x', 'y']].head(2)

,x,y
0,"[395534.70260021265, 394047.81013838784, 39396...","[6679490.084630681, 6679228.490560529, 6679185..."
1,"[384398.5021810767, 384626.19157614314, 384746...","[6671336.407362772, 6671348.068805486, 6671365..."


In [32]:
points[['x', 'y']].head(2)

,x,y
0,385149.447837,6.671963e+06
1,385804.985388,6.672109e+06


- Let’s now classify the travel times of our grid int 5 minute intervals until 200 minutes using a pysal classifier called User_Defined that allows to set our own criteria for class intervals. But first we need to replace the No Data values with a large number so that they wouldn’t be seen as the “best” accessible areas.

In [36]:
# Replace No Data values (-1) with large number (999)
grid = grid.replace(-1, 999)

# Classify our travel times into 5 minute classes until 200 minutes
# Create a list of values where minumum value is 5, maximum value is 200 and step is 5.
breaks = [x for x in range(5, 200, 5)]

# Initialize the classifier and apply it
classifier = ps.User_Defined.make(bins=breaks)
pt_classif = grid[['pt_r_tt']].apply(classifier)

# Rename the classified column
pt_classif.columns = ['pt_r_tt_ud']

# Join it back to the grid layer
grid = grid.join(pt_classif)


In [38]:
grid.head(2)

,car_m_d,car_m_t,car_r_d,car_r_t,from_id,pt_m_d,pt_m_t,pt_m_tt,pt_r_d,pt_r_t,pt_r_tt,to_id,walk_d,walk_t,geometry,x,y,pt_r_tt_ud
0,32297,43,32260,48,5785640,32616,116,147,32616,108,139,5975375,32164,459,"POLYGON ((382000.0001358641 6697750.000038058,...","[382000.00013586413, 381750.0001359122, 381750...","[6697750.000038058, 6697750.000038066, 6698000...",27
1,32508,43,32471,49,5785641,32822,119,145,32822,111,133,5975375,29547,422,"POLYGON ((382250.0001358146 6697750.000038053,...","[382250.0001358146, 382000.00013586413, 382000...","[6697750.000038053, 6697750.000038058, 6698000...",26


- Okey, so we have many columns but the new one that we just got is the last one, i.e. pt_r_tt_ud that contains the classes that we reclassified based on the public transportation travel times on 5 minute intervals.

In [39]:
# Make a copy, drop the geometry column and create ColumnDataSource
m_df = metro.drop('geometry', axis=1).copy()
msource = ColumnDataSource(m_df)

# Make a copy, drop the geometry column and create ColumnDataSource
p_df = points.drop('geometry', axis=1).copy()
psource = ColumnDataSource(p_df)

# Make a copy, drop the geometry column and create ColumnDataSource
g_df = grid.drop('geometry', axis=1).copy()
gsource = ColumnDataSource(g_df)

> http://bokeh.pydata.org/en/latest/docs/reference/palettes.html#bokeh-palettes
> - **4th step**: For visualizing the Polygons we need to define the color palette that we are going to use. There are many different ones available but we are now going to use a palette called RdYlBu and use eleven color-classes for the values (defined as RdYlBu11). Let’s prepare our color_mapper.

In [40]:
# Let's first do some coloring magic that converts the color palet into map numbers (it's okey not to understand)
from bokeh.palettes import RdYlBu11 as palette
from bokeh.models import LogColorMapper

# Create the color mapper
color_mapper = LogColorMapper(palette=palette)

In [41]:
# Initialize our figure
p = figure(title="Travel times with Public transportation to Central Railway station")

# Plot grid
p.patches('x', 'y', source=gsource,
         fill_color={'field': 'pt_r_tt_ud', 'transform': color_mapper},
         fill_alpha=1.0, line_color="black", line_width=0.05)

# Add metro on top of the same figure
p.multi_line('x', 'y', source=msource, color="red", line_width=2)

# Add points on top (as black points)
p.circle('x', 'y', size=3, source=psource, color="black")

# Save the figure
outfp = './shapely/dataE5/dataE5/travel_time_map.html'
save(p, outfp)

/home/dockeruser/anaconda3/lib/python3.6/site-packages/bokeh/io.py:419: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warnings.warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/home/dockeruser/anaconda3/lib/python3.6/site-packages/bokeh/io.py:429: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warnings.warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'/home/dockeruser/hostname/workspace/git/kaden/individual_project_20171130/shapely/dataE5/dataE5/travel_time_map.html'

- Visualizaing details : https://automating-gis-processes.github.io/Lesson5-interactive-map-Bokeh-advanced-plotting.html

# Interactive maps on Leaflet

- Whenever you go into a website that has some kind of interactive map, it is quite probable that you are wittnessing a map that has been made with a JavaScipt library called Leaflet (the other popular one that you might have wittnessed is called OpenLayers).
- Leaflet : http://leafletjs.com/
- OpenLayers : https://openlayers.org/
- Folium : https://github.com/python-visualization/folium

In [43]:
import folium

In [56]:
map_osm = folium.Map(location=[60.25, 24.8], tiles='Stamen Toner',
                   zoom_start=10, control_scale=True)

> The first parameter location takes a pair of lat, lon values as list as an input which will determine where the map will be positioned when user opens up the map. zoom_start -parameter adjusts the default zoom-level for the map (the higher the number the closer the zoom is). control_scale defines if map should have a scalebar or not.

In [45]:
outfp = "./shapely/Processed_data/base_map.html"
m.save(outfp)

In [46]:
# Let's change the basemap style to 'Stamen Toner'
m = folium.Map(location=[37.54205, 126.86598], tiles='Stamen Toner',
                zoom_start=12, control_scale=True, prefer_canvas=True)

# Filepath to the output
outfp = "./shapely/Processed_data/base_map_seoul.html"

# Save the map
m.save(outfp)

In [49]:
from fiona.crs import from_epsg

# Filepaths
fp = "./shapely/Vaestotietoruudukko_2015/Vaestotietoruudukko_2015.shp"
addr_fp = "./shapely/dataE5/dataE5/addresses.shp"

# Read Data
data = gpd.read_file(fp)
ad = gpd.read_file(addr_fp)

# Re-project to WGS84
data['geometry'] = data['geometry'].to_crs(epsg=4326)
ad['geometry'] = ad['geometry'].to_crs(epsg=4326)

# Update the CRS of the GeoDataFrame
data.crs = from_epsg(4326)
ad.crs = from_epsg(4326)

# Make a selection (only data above 0 and below 1000)
data = data.loc[(data['ASUKKAITA'] > 0) & (data['ASUKKAITA'] <= 1000)]

# Create a Geo-id which is needed by the Folium (it needs to have a unique identifier for each row)
data['geoid'] = data.index.astype(str)
ad['geoid'] = ad.index.astype(str)

# Select data
data = data[['geoid', 'ASUKKAITA', 'geometry']]

# Save the file as geojson
jsontxt = data.to_json()

INFO:Fiona:Failed to auto identify EPSG: 7


> - Now we have our population data stored in the jsontxt variable as GeoJSON format which basically contains the data as text in a similar way that it would be written in the .geojson -file.

Now we can start visualizing our data with Folium.

In [51]:
folium.__version__

'0.5.0'

In [55]:
data

,geoid,ASUKKAITA,geometry
0,0,8,"POLYGON ((24.50236241370834 60.31927864851716,..."
1,1,6,"POLYGON ((24.50287385337343 60.28562263749414,..."
2,2,8,"POLYGON ((24.50311210582754 60.26991652312412,..."
3,3,7,"POLYGON ((24.50314612020954 60.26767278939882,..."
4,4,19,"POLYGON ((24.50328212493893 60.25869775697638,..."
5,5,11,"POLYGON ((24.50735919320162 60.28788319783227,..."
6,6,6,"POLYGON ((24.50739294139509 60.28563946878506,..."
7,7,22,"POLYGON ((24.50762902824739 60.26993334375615,..."
8,8,21,"POLYGON ((24.50779750196551 60.2587145700011, ..."
9,9,9,"POLYGON ((24.50783118034555 60.25647083087788,..."


In [59]:
from folium.plugins import MarkerCluster

# Create a Clustered map where points are clustered
marker_cluster = MarkerCluster().add_to(map_osm)


# Create Choropleth map where the colors are coming from a column "ASUKKAITA".
# Notice: 'geoid' column that we created earlier needs to be assigned always as the first column
# with threshold_scale we can adjust the class intervals for the values
map_osm.choropleth(geo_data=jsontxt, data=data, columns=['geoid', 'ASUKKAITA'], key_on="feature.id",
                   fill_color='YlOrRd', fill_opacity=0.9, line_opacity=0.2, line_color='white', line_weight=0,
                   threshold_scale=[100, 250, 500, 1000, 2000],
                   legend_name='Population in Helsinki', highlight=False, smooth_factor=1.0)


# Create Address points on top of the map
for idx, row in ad.iterrows():
    # Get lat and lon of points
    lon = row['geometry'].x
    lat = row['geometry'].y

    # Get address information
    address = row['address']
    # Add marker to the map
    folium.RegularPolygonMarker(location=[lat, lon], popup=address, fill_color='#2b8cbe', number_of_sides=6, radius=8).add_to(marker_cluster)

# Save the output
outfp = r"./shapely/Processed_data/pop15.html"
map_osm.save(outfp)

### 3D Folium Example

In [60]:
from folium.features import Template


class Map3d(folium.Map):

    def __init__(self, location=None, width='100%', height='100%', left='0%',
                 top='0%', position='relative', tiles='OpenStreetMap', API_key=None,
                 max_zoom=18, min_zoom=1, zoom_start=10, attr=None, min_lat=-90,
                 max_lat=90, min_lon=-180, max_lon=180, detect_retina=False, crs='EPSG3857'):
        super(Map3d, self).__init__(
            location=location, width=width, height=height,
            left=left, top=top, position=position, tiles=tiles,
            API_key=API_key, max_zoom=max_zoom, min_zoom=min_zoom,
            zoom_start=zoom_start, attr=attr, min_lat=min_lat,
            max_lat=max_lat, min_lon=min_lon, max_lon=max_lon,
            detect_retina=detect_retina, crs=crs
        )
        self._template = Template(u"""
        {% macro header(this, kwargs) %}
            <script src="https://www.webglearth.com/v2/api.js"></script>
            <style> #{{this.get_name()}} {
                position : {{this.position}};
                width : {{this.width[0]}}{{this.width[1]}};
                height: {{this.height[0]}}{{this.height[1]}};
                left: {{this.left[0]}}{{this.left[1]}};
                top: {{this.top[0]}}{{this.top[1]}};
                }
            </style>
        {% endmacro %}
        {% macro html(this, kwargs) %}
            <div class="folium-map" id="{{this.get_name()}}" ></div>
        {% endmacro %}

        {% macro script(this, kwargs) %}

            var southWest = L.latLng({{ this.min_lat }}, {{ this.min_lon }});
            var northEast = L.latLng({{ this.max_lat }}, {{ this.max_lon }});
            var bounds = L.latLngBounds(southWest, northEast);

            var {{this.get_name()}} = WE.map('{{this.get_name()}}', {
                                           center:[{{this.location[0]}},{{this.location[1]}}],
                                           zoom: {{this.zoom_start}},
                                           maxBounds: bounds,
                                           layers: [],
                                           crs: L.CRS.{{this.crs}}
                                         });
        {% endmacro %}
        """)


class TileLayer3d(folium.TileLayer):

    def __init__(self, tiles='OpenStreetMap', min_zoom=1, max_zoom=18, attr=None,
                 API_key=None, detect_retina=False, name=None, overlay=False, control=True):
        super(TileLayer3d, self).__init__(
            tiles=tiles, min_zoom=min_zoom, max_zoom=max_zoom,
            attr=attr, API_key=API_key, detect_retina=detect_retina,
            name=name, overlay=overlay, control=control
        )
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            var {{this.get_name()}} = WE.tileLayer(
                '{{this.tiles}}',
                {
                    maxZoom: {{this.max_zoom}},
                    minZoom: {{this.min_zoom}},
                    attribution: '{{this.attr}}',
                    detectRetina: {{this.detect_retina.__str__().lower()}}
                    }
                ).addTo({{this._parent.get_name()}});

        {% endmacro %}
        """)

In [62]:
import os

In [63]:
m = Map3d(location=[-42, -42], tiles=None, zoom_start=2)
m.add_child(TileLayer3d(tiles='OpenStreetMap'))

url = ('http://services.arcgisonline.com/arcgis/rest/services'
       'Ocean/World_Ocean_Base'
       'MapServer/tile/{z}/{y}/{x}')

folium.TileLayer(
    tiles=url,
    name='World_Ocean_Base',
    attr='ESRI',
    overlay=True
).add_to(m)


m.save(os.path.join('./shapely', 'Hacking_folium_3D_globe.html'))

# Lesson 6 
## Objectives
- About Arcpy

# Lesson 7 
- In this lesson we will learn basic techniques for automating raster data processing with Python and the Geospatial Data Abstraction Library GDAL.
- http://www.gdal.org/
- Raster files are commonly used to store terrain models and remote sensing data and their derivate products such as vegetation indices and other environmental datasets. Raster files tend to be huge (imagine for example a raster dataset covering the globe in 30m x 30m resolution) and are often delivered and processed in smaller pieces (tiles). Efficient processing and analysis of such large datasets consequently requires automatization. During this lesson you will learn how to read and write common raster formats, and conduct basic raster data processes for a batch of files using the GDAL/OGR API in Python and GDAL command line utilities.
- https://mapbox.github.io/rasterio/


### Learning Objectives
- Read / write raster data (e.g. .tif) in Python
- Extract some basic raster statistics from the data (min, max, mean, no-data-value etc.)
- Clip the raster dataset by bounding box
- Stack different bands together and create a false-color composite
- Calculate with rasters

- With GDAL, you can read and write several different raster formats in Python. Python automatically registers all known GDAL drivers for reading supported formats when the importing the GDAL module. Most common file formats include for example TIFF and GeoTIFF, ASCII Grid and Erdas Imagine .img -files.

In [4]:
from gdal import gdal

filepath = r"./shapely/LC81910182016153-SC20161208043748/LC81910182016153LGN00_sr_band4.tif"

# Open the file:
raster = gdal.Open(filepath)

# Check type of the variable 'raster'
type(raster)

ModuleNotFoundError: No module named 'gdal'

In [5]:
from osgeo import gdal, ogr

ModuleNotFoundError: No module named 'osgeo'